# Combining tides with satellite data

This guide demonstrates how to combine tide modelling with satellite Earth observation (EO) data using the [`tag_tides`](../../api/#eo_tides.eo.tag_tides) and [`pixel_tides`](../../api/#eo_tides.eo.pixel_tides) functions from [`eo_tides.eo`](../../api/#eo_tides.eo). 

Both these functions allow you to model the height of the tide at the exact moment of satellite image acquisition. 
This can then allow you to analyse satellite EO data by tidal conditions - for example, filter your data to satellite imagery collected during specific tidal stages (e.g. low or high tide).

Although both functions perform a similar function, they differ in complexity and performance. `tag_tides` assigns a single tide height to each timestep/satellite image, which is fast and efficient, and suitable for small-scale applications where tides are unlikely to vary across your study area. In constrast, `pixel_tide` models tides both through time *and* spatially, returning a tide height for every satellite pixel. This can be critical for producing seamless coastal EO datasets at large scale - however comes at the cost of performance.
<br><br>
**Comparison of `tag_tides` and `pixel_tides`**
| [`tag_tides`](../../api/#eo_tides.eo.tag_tides)                                                                 | [`pixel_tides`](../../api/#eo_tides.eo.pixel_tides)                                                                                              |
|-----------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------|
| Assigns a single tide height to each timestep/satellite image                         | Assigns a tide height to every individual pixel through time                                               |
| Ideal for local or site-scale analysis                                      | Ideal for regional to global-scale coastal product generation                                              |
| ✅ Fast, low memory use                                                        | ❌ Slower, higher memory use                                                                                  |
| ❌ Single tide height per image can produce artefacts in complex tidal regions | ✅ Can produce spatially seamless outputs across large extents by applying analyses at the pixel level |

## Load satellite data using odc-stac

First, we will load a time-series of satellite data over our area of interest using the Open Data Cube's `odc-stac` package.
This powerful package allows us to load open satellite data (e.g ESA Sentinel-2 or NASA/USGS Landsat) for any time period and location on the planet, and load our data into a multi-dimensional `xarray.Dataset` format dataset.

In this example, we will load **Landsat 8 and 9** satellite data from **January 2024** over the city of **Broome, Western Australia** - a region with a macrotidal tide range and extensive intertidal coastal habitats.

<div class="admonition tip">
    <p class="admonition-title">Tip</p>
    <p>
        For a more detailed guide to using STAC metadata and <code>odc-stac</code> to find and load satellite data, refer to the <a href="https://knowledge.dea.ga.gov.au/guides/setup/gis/stac/">Digital Earth Australia STAC user guide</a>.
    </p>
</div>

In [9]:
import odc.stac
import pystac_client

# Connect to STAC catalog
catalog = pystac_client.Client.open("https://explorer.dea.ga.gov.au/stac")

# Set cloud defaults
odc.stac.configure_rio(
    cloud_defaults=True,
    aws={"aws_unsigned": True},
)

# Build a query with the parameters above
bbox = [122.160, -18.05, 122.260, -17.95]
query = catalog.search(
    bbox=bbox,
    collections=["ga_ls8c_ard_3", "ga_ls9c_ard_3"],
    datetime="2024-01-01/2024-01-31",
)

# Search the STAC catalog for all items matching the query
ds = odc.stac.load(
    list(query.items()),
    bands=["nbart_red", "nbart_green", "nbart_blue"],
    crs="utm",
    resolution=30,
    groupby="solar_day",
    bbox=bbox,
    fail_on_error=False,
    chunks={},
)

ds.isel(time=0).odc.explore(vmin=0, vmax=2000)

## Using tag_tides

We can pass our satellite dataset `ds` to the `tag_tides` function to model a tide for each timestep in our dataset:

In [10]:
from eo_tides.eo import tag_tides

ds = tag_tides(
    ds=ds,
    model="EOT20",
    directory="../../tests/data/tide_models"
)

Setting tide modelling location from dataset centroid: 122.21, -18.00
Modelling tides using EOT20


Our output data now has a new "tide_height" variable under **Data variables**:

In [11]:
print(ds)

<xarray.Dataset> Size: 11MB
Dimensions:      (y: 371, x: 356, time: 7)
Coordinates:
  * y            (y) float64 3kB 8.015e+06 8.015e+06 ... 8.004e+06 8.004e+06
  * x            (x) float64 3kB 4.11e+05 4.111e+05 ... 4.217e+05 4.217e+05
    spatial_ref  int32 4B 32751
  * time         (time) datetime64[ns] 56B 2024-01-07T01:55:31.679580 ... 202...
    tide_model   <U5 20B 'EOT20'
Data variables:
    nbart_red    (time, y, x) float32 4MB dask.array<chunksize=(1, 371, 356), meta=np.ndarray>
    nbart_green  (time, y, x) float32 4MB dask.array<chunksize=(1, 371, 356), meta=np.ndarray>
    nbart_blue   (time, y, x) float32 4MB dask.array<chunksize=(1, 371, 356), meta=np.ndarray>
    tide_height  (time) float32 28B -0.2641 0.5031 -0.8923 ... 1.658 -1.237


## Using pixel_tides



In [12]:
from eo_tides.eo import pixel_tides

tides_highres, tides_lowres = pixel_tides(
    ds=ds,
    model="EOT20",
    directory="../../tests/data/tide_models"
)

tides_highres.isel(time=0).odc.explore()

Creating reduced resolution 5000 x 5000 metre tide modelling array
Modelling tides using EOT20 in parallel


 20%|██        | 1/5 [00:00<00:00,  6.16it/s]

100%|██████████| 5/5 [00:00<00:00, 10.94it/s]


Reprojecting tides into original array


## Next steps

Now that we have learnt to combine tide modelling with satellite data, we can learn how to [calculate statistics](../Tide_statistics) describing local tide dynamics, as well as biases caused by interactions between tidal processes and satellite orbits.